<a href="https://colab.research.google.com/github/akimi-yano/Kenyan_Cuisine_Image_Classification/blob/main/Kenyan_Cuisine_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [62]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


Kaggle credentials set.
Kaggle credentials successfully validated.


In [33]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

opencv_pytorch_project_2_classification_round_3_path = kagglehub.competition_download('opencv-pytorch-project-2-classification-round-3')

print('Data source import complete.')


Data source import complete.


In [63]:
from google.colab import files
files.upload()

{}

In [35]:
!mkdir -p ~/.kaggle

In [36]:
! cp kaggle.json ~/.kaggle/

In [37]:
!ls ~/.kaggle

kaggle.json


In [38]:
!chmod 600 /root/.kaggle/kaggle.json

In [39]:
! kaggle competitions download -c opencv-pytorch-project-2-classification-round-3

opencv-pytorch-project-2-classification-round-3.zip: Skipping, found more recently modified local copy (use --force to force download)


In [40]:
!unzip opencv-pytorch-project-2-classification-round-3.zip -d dataset/

Streaming output truncated to the last 5000 lines.
  inflating: dataset/images/images/16273186685497510505.jpg  
  inflating: dataset/images/images/16282614765807136033.jpg  
  inflating: dataset/images/images/16284339368408617325.jpg  
  inflating: dataset/images/images/16291671383938929277.jpg  
  inflating: dataset/images/images/16292068910159576399.jpg  
  inflating: dataset/images/images/16296225045024370393.jpg  
  inflating: dataset/images/images/16299062930688223105.jpg  
  inflating: dataset/images/images/16299107618575423489.jpg  
  inflating: dataset/images/images/16303389888524426910.jpg  
  inflating: dataset/images/images/16304405155635632552.jpg  
  inflating: dataset/images/images/1630480295804231927.jpg  
  inflating: dataset/images/images/16305353331382847428.jpg  
  inflating: dataset/images/images/16310534578938153796.jpg  
  inflating: dataset/images/images/16316726223246099607.jpg  
  inflating: dataset/images/images/16321027829450957963.jpg  
  inflating: dataset

In [41]:
import os
data_root = './dataset'
os.path.exists(data_root)
for elem in os.walk(data_root):
  print(elem)

('./dataset', ['images'], ['train.csv', 'sample_submission.csv', 'test.csv'])
('./dataset/images', ['images'], [])
('./dataset/images/images', [], ['6636759988962883605.jpg', '8362038071878427831.jpg', '6572056070648508463.jpg', '14930267842709174527.jpg', '16650682517014422364.jpg', '8481171762791803367.jpg', '3603542422109599326.jpg', '11450733710039892265.jpg', '15181524804269396907.jpg', '9209822241129633192.jpg', '3529354745490544317.jpg', '379145191247681558.jpg', '8949452682245057130.jpg', '6038664787270631453.jpg', '11351098185926786646.jpg', '9737513360498481843.jpg', '6582452949275389646.jpg', '17989123767488385348.jpg', '8424682282407494386.jpg', '5274203241956843203.jpg', '761270166752000015.jpg', '4618285748886797950.jpg', '1285191645597279119.jpg', '8446705810507569427.jpg', '17114243994256232408.jpg', '4617601781150446290.jpg', '12127185769394707033.jpg', '14778322540850194798.jpg', '13953013292218577724.jpg', '672954769923018165.jpg', '14815644896028333872.jpg', '157935

## <font style="color:green">Data Loader</font>

In [42]:
import os
import pandas as pd
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
from PIL import Image
import torch

In [43]:
label_mapping = {
            "pilau": 0,
            "nyamachoma": 1,
            "masalachips": 2,
            "mandazi": 3,
            "ugali": 4,
            "sukumawiki": 5,
            "mukimo": 6,
            "githeri": 7,
            "matoke": 8,
            "kukuchoma": 9,
            "chapati": 10,
            "kachumbari": 11,
            "bhaji": 12,
        }

In [44]:
class KenyanFood13Dataset(Dataset):
    """
    Custom dataset class for KenyanFood13 that handles both train and test datasets.

    Args:
        csv_file (str): Path to the CSV file containing image paths and optionally labels.
        img_dir (str): Path to the directory containing images.
        transform (callable, optional): Transformation function to apply to the images.
    """

    def __init__(self, csv_file, img_dir, transform=None, has_labels=False):
        self.data = pd.read_csv(csv_file)
        self.img_dir = img_dir
        self.transform = transform
        self.has_labels = has_labels
        self.label_mapping = label_mapping

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # Extract image ID and construct the file path with .jpg extension
        img_id = str(self.data.iloc[idx, 0])  # Assuming first column is image ID
        img_path = os.path.join(self.img_dir, img_id + ".jpg")

        # Load image
        image = Image.open(img_path).convert("RGB")

        # Apply transformations
        if self.transform:
            image = self.transform(image)

        # Return label if available
        if self.has_labels:
            label = str(self.data.iloc[idx, 1])  # Assuming second column is the label
            label = self.label_mapping[label]   # Map label to integer
            label = torch.tensor(label, dtype=torch.long)  # Convert to tensor

            return image, label
        else:
            return image, img_id  # Return image and ID for test data


In [45]:
def get_data_loaders(train_csv, img_dir, batch_size=8, val_split=0.2):
    """
    Creates data loaders for train and validation datasets.

    Args:
        train_csv (str): Path to the train.csv file.
        img_dir (str): Path to the directory containing images.
        batch_size (int): Batch size for data loaders.
        val_split (float): Fraction of data to use for validation.

    Returns:
        train_loader (DataLoader): DataLoader for training data.
        val_loader (DataLoader): DataLoader for validation data.
    """

    # Define data transformations
    train_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(20),
        transforms.ColorJitter(0.3, 0.3, 0.3),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

    val_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

    # Load the entire dataset
    full_dataset = KenyanFood13Dataset(csv_file=train_csv, img_dir=img_dir, transform=None, has_labels=True)

    # Split dataset into train and validation sets
    val_size = int(len(full_dataset) * val_split)
    train_size = len(full_dataset) - val_size
    train_dataset, val_dataset = torch.utils.data.random_split(full_dataset, [train_size, val_size])

    # Assign transformations to train and validation datasets
    train_dataset.dataset.transform = train_transform
    val_dataset.dataset.transform = val_transform

    # Create data loaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

    return train_loader, val_loader

In [46]:
def get_test_loader(test_csv, img_dir, batch_size=8):
    """
    Prepares a data loader for test data.

    Args:
        test_csv (str): Path to test.csv containing image filenames (no labels).
        img_dir (str): Path to the directory containing images.
        batch_size (int): Batch size for the test data loader.

    Returns:
        test_loader (DataLoader): DataLoader for test data.
    """
    # Define data transformations (same as training, except no labels required)
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    # Create the test dataset
    test_dataset = KenyanFood13Dataset(csv_file=test_csv, img_dir=img_dir, transform=transform, has_labels=False)

    # Create test data loader
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

    return test_loader

In [47]:
train_csv = "./dataset/train.csv"
test_csv = "./dataset/test.csv"
img_dir = "./dataset/images/images"
train_loader, val_loader = get_data_loaders(train_csv, img_dir, batch_size=8)
test_loader = get_test_loader(test_csv, img_dir, batch_size=8)

## <font style="color:green">Configuration</font>

In [48]:
from dataclasses import dataclass

In [49]:
@dataclass
class TrainingConfiguration:
    '''
    Describes configuration of the training process
    '''
    batch_size: int = 8
    epochs_count: int = 50
    init_learning_rate: float = 0.0001  # initial learning rate for lr scheduler
    log_interval: int = 5
    test_interval: int = 1
    data_root: str = img_dir
    num_workers: int = 2
    device: str = 'cuda'
    num_classes: int = 13
    patience: int = 5

## <font style="color:green">Evaluation Metric</font>

In [50]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report
import torch

In [51]:
class EvaluationMetrics:
    """
    A class to compute evaluation metrics for classification tasks.
    """

    @staticmethod
    def calculate_metrics(y_true, y_pred, average="macro"):
        """
        Calculate and return evaluation metrics.

        Args:
            y_true (list or numpy array): Ground truth labels.
            y_pred (list or numpy array): Predicted labels.
            average (str): Averaging method for metrics (e.g., 'macro', 'micro', 'weighted').

        Returns:
            dict: Dictionary containing accuracy, precision, recall, and F1-score.
        """
        metrics = {
            "accuracy": accuracy_score(y_true, y_pred),
            "precision": precision_score(y_true, y_pred, average=average, zero_division=1),
            "recall": recall_score(y_true, y_pred, average=average, zero_division=1),
            "f1_score": f1_score(y_true, y_pred, average=average, zero_division=1),
        }
        return metrics

    @staticmethod
    def print_classification_report(y_true, y_pred, target_names=None):
        """
        Print the detailed classification report.

        Args:
            y_true (list or numpy array): Ground truth labels.
            y_pred (list or numpy array): Predicted labels.
            target_names (list, optional): Names of classes.
        """
        report = classification_report(y_true, y_pred, target_names=target_names, zero_division=1)
        print(report)

    @staticmethod
    def evaluate_model(model, data_loader, device="cuda"):
        """
        Evaluate the model on a given dataset.

        Args:
            model (torch.nn.Module): Trained PyTorch model.
            data_loader (torch.utils.data.DataLoader): DataLoader for evaluation.
            device (str): Device to run the evaluation on ('cuda' or 'cpu').

        Returns:
            dict: Dictionary containing metrics (accuracy, precision, recall, F1-score).
        """
        model.eval()
        y_true = []
        y_pred = []

        with torch.no_grad():
            for batch in data_loader:
                images, labels = batch
                images = images.to(device)
                labels = labels.to(device)

                outputs = model(images)
                _, predictions = torch.max(outputs, 1)

                y_true.extend(labels.cpu().numpy())
                y_pred.extend(predictions.cpu().numpy())

        metrics = EvaluationMetrics.calculate_metrics(y_true, y_pred)
        metrics.update({"y_true": y_true, "y_pred": y_pred})
        return metrics


## <font style="color:green">Train and Validation</font>

In [52]:
import torch
import torch.nn as nn
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter

class Trainer:
    """
    A class to handle training and validation of a PyTorch model with TensorBoard integration.
    """

    def __init__(self, model, optimizer, criterion, lr_scheduler, device="cuda", log_dir="runs/project_logs", ):
        """
        Initializes the Trainer class.

        Args:
            model (torch.nn.Module): The PyTorch model to be trained and validated.
            optimizer (torch.optim.Optimizer): Optimizer for updating model weights.
            criterion (torch.nn.Module): Loss function.
            device (str): Device to run training and validation on ('cuda' or 'cpu').
            log_dir (str): Directory to save TensorBoard logs.
        """
        self.device = device
        self.model = model.to(self.device)
        self.criterion = criterion.to(self.device)
        self.optimizer = optimizer
        self.lr_scheduler = lr_scheduler

        # Initialize TensorBoard writer
        self.writer = SummaryWriter(log_dir=log_dir)

    def train_one_epoch(self, train_loader, epoch):
        """
        Train the model for one epoch.

        Args:
            train_loader (torch.utils.data.DataLoader): DataLoader for training data.
            epoch (int): Current epoch number.

        Returns:
            float: Average training loss for the epoch.
        """
        self.model.train()
        epoch_loss = 0.0

        # Loop over batches
        for batch_idx, (images, labels) in enumerate(tqdm(train_loader, desc=f"Training Epoch {epoch}")):
            # print(type(images), type(labels))
            images = images.to(self.device)
            labels = labels.to(self.device)

            # Zero the gradients
            self.optimizer.zero_grad()

            # Forward pass
            outputs = self.model(images)

            # Compute loss
            loss = self.criterion(outputs, labels)

            # Backpropagation
            loss.backward()

            # Update weights
            self.optimizer.step()

            # Accumulate loss
            epoch_loss += loss.item()

            # Log batch loss
            self.writer.add_scalar('Loss/Train_Batch', loss.item(), epoch * len(train_loader) + batch_idx)

        return epoch_loss / len(train_loader)

    def validate(self, val_loader, epoch):
        """
        Validate the model on the validation dataset.

        Args:
            val_loader (torch.utils.data.DataLoader): DataLoader for validation data.
            epoch (int): Current epoch number.

        Returns:
            dict: Validation loss and evaluation metrics (accuracy, precision, recall, F1-score).
        """
        self.model.eval()
        val_loss = 0.0
        y_true = []
        y_pred = []

        with torch.no_grad():
            for images, labels in tqdm(val_loader, desc=f"Validation Epoch {epoch}"):
                images = images.to(self.device)
                labels = labels.to(self.device)

                # Forward pass
                outputs = self.model(images)

                # Compute loss
                loss = self.criterion(outputs, labels)
                val_loss += loss.item()

                # Predictions
                _, predictions = torch.max(outputs, 1)
                y_true.extend(labels.cpu().numpy())
                y_pred.extend(predictions.cpu().numpy())

        # Compute evaluation metrics
        metrics = EvaluationMetrics.calculate_metrics(y_true, y_pred)
        metrics["loss"] = val_loss / len(val_loader)

        # Log validation metrics to TensorBoard
        self.writer.add_scalar('Loss/Validation', metrics["loss"], epoch)
        self.writer.add_scalar('Accuracy/Validation', metrics["accuracy"], epoch)
        self.writer.add_scalar('Precision/Validation', metrics["precision"], epoch)
        self.writer.add_scalar('Recall/Validation', metrics["recall"], epoch)
        self.writer.add_scalar('F1_Score/Validation', metrics["f1_score"], epoch)

        return metrics

    def train_and_validate(self, train_loader, val_loader, config):
        """
        Train and validate the model for multiple epochs.

        Args:
            train_loader (torch.utils.data.DataLoader): DataLoader for training data.
            val_loader (torch.utils.data.DataLoader): DataLoader for validation data.
            config (TrainingConfiguration): Configuration for training (batch size, epochs, etc.).

        Returns:
            None
        """
        best_val_loss = float('inf')
        patience = config.patience  # Number of epochs to wait before stopping
        patience_counter = 0

        for epoch in range(config.epochs_count):
            print(f"\nEpoch {epoch + 1}/{config.epochs_count}")

            # Training phase
            train_loss = self.train_one_epoch(train_loader, epoch)
            print(f"Training Loss: {train_loss:.4f}")
            self.writer.add_scalar('Loss/Train', train_loss, epoch)

            # Validation phase
            metrics = self.validate(val_loader, epoch)
            val_loss = metrics["loss"]
            print(f"Validation Metrics: {metrics}")

            # Check if the current model is the best so far
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                patience_counter = 0  # Reset counter
                torch.save(self.model.state_dict(), 'best_model.pth')
                print(f"Best model saved with loss: {best_val_loss:.4f}")
            else:
                patience_counter += 1
                print(f"No improvement for {patience_counter} epoch(s).")

            # Check if patience has been exceeded
            if patience_counter >= patience:
                print("Early stopping triggered. Training terminated.")
                break

            self.lr_scheduler.step()

        # Close the TensorBoard writer after training
        self.writer.close()

## <font style="color:green">Model</font>

In [53]:
import timm
print(timm.list_models("*convnext*"))

['convnext_atto', 'convnext_atto_ols', 'convnext_atto_rms', 'convnext_base', 'convnext_femto', 'convnext_femto_ols', 'convnext_large', 'convnext_large_mlp', 'convnext_nano', 'convnext_nano_ols', 'convnext_pico', 'convnext_pico_ols', 'convnext_small', 'convnext_tiny', 'convnext_tiny_hnf', 'convnext_xlarge', 'convnext_xxlarge', 'convnext_zepto_rms', 'convnext_zepto_rms_ols', 'convnextv2_atto', 'convnextv2_base', 'convnextv2_femto', 'convnextv2_huge', 'convnextv2_large', 'convnextv2_nano', 'convnextv2_pico', 'convnextv2_small', 'convnextv2_tiny', 'test_convnext', 'test_convnext2', 'test_convnext3']


In [54]:
import torch.nn as nn
import torchvision.models as models
import timm

class KenyanFood13Model(nn.Module):
    """
    A classification model based on a pre-trained ResNet18.
    """

    # convnext_base
    # swin_base_patch4_window7_224
    # convnext_small

    # def __init__(self, num_classes, model_name="convnext_base"):
    def __init__(self, num_classes, model_name="swin_base_patch4_window7_224"):
        """
        Initializes the KenyanFood13Model.

        Args:
            num_classes (int): The number of output classes for classification.
        """
        super(KenyanFood13Model, self).__init__()

        # Load the pre-trained model from timm
        self.base_model = timm.create_model(model_name, pretrained=True)
        # Get the number of input features for the classifier
        in_features = self.base_model.get_classifier().in_features
        # Replace the classifier with a custom layer for the desired number of classes
        self.base_model.reset_classifier(num_classes)

    def forward(self, x):
        """
        Forward pass of the model.

        Args:
            x (torch.Tensor): Input image tensor.

        Returns:
            torch.Tensor: Output logits for each class.
        """
        return self.base_model(x)


## <font style="color:green">Utils</font>

In [55]:
def create_reverse_mapping(label_mapping):
    """
    Create a reverse mapping from a given label mapping dictionary.

    Args:
        label_mapping (dict): A dictionary where keys are original labels and values are mapped labels.

    Returns:
        dict: A dictionary where keys are the mapped labels and values are the original labels.
    """
    reverse_label_map = {v: k for k, v in label_mapping.items()}
    return reverse_label_map

In [56]:
reverse_label_map = create_reverse_mapping(label_mapping)
print(label_mapping)
print(reverse_label_map)

{'pilau': 0, 'nyamachoma': 1, 'masalachips': 2, 'mandazi': 3, 'ugali': 4, 'sukumawiki': 5, 'mukimo': 6, 'githeri': 7, 'matoke': 8, 'kukuchoma': 9, 'chapati': 10, 'kachumbari': 11, 'bhaji': 12}
{0: 'pilau', 1: 'nyamachoma', 2: 'masalachips', 3: 'mandazi', 4: 'ugali', 5: 'sukumawiki', 6: 'mukimo', 7: 'githeri', 8: 'matoke', 9: 'kukuchoma', 10: 'chapati', 11: 'kachumbari', 12: 'bhaji'}


## <font style="color:green">Experiment</font>

In [57]:
# from torch.optim import Adam
from torch.optim import SGD, lr_scheduler
from torch.nn import CrossEntropyLoss

# Define model, optimizer, and loss function
model = KenyanFood13Model(TrainingConfiguration.num_classes)
optimizer = SGD(model.parameters(), lr=0.0001, momentum=0.9)
criterion = CrossEntropyLoss()
lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

# Initialize Trainer
trainer = Trainer(model=model, optimizer=optimizer, lr_scheduler=lr_scheduler, criterion=criterion, device="cuda")

In [58]:
trainer.train_and_validate(train_loader, val_loader, TrainingConfiguration())


Epoch 1/50


Training Epoch 0: 100%|██████████| 654/654 [01:34<00:00,  6.91it/s]


Training Loss: 1.5741


Validation Epoch 0: 100%|██████████| 164/164 [00:08<00:00, 19.28it/s]


Validation Metrics: {'accuracy': 0.7299158377964805, 'precision': 0.7703351994771807, 'recall': 0.6508700373852105, 'f1_score': 0.6487925445801334, 'loss': 0.8877515306287422}
Best model saved with loss: 0.8878

Epoch 2/50


...

Validation Metrics: {'accuracy': 0.8247895944912013, 'precision': 0.8068004560427673, 'recall': 0.7850619700554252, 'f1_score': 0.7917893422433674, 'loss': 0.5190163614887108}
No improvement for 4 epoch(s).

Epoch 16/50


Training Epoch 15: 100%|██████████| 654/654 [01:36<00:00,  6.78it/s]


Training Loss: 0.2220


Validation Epoch 15: 100%|██████████| 164/164 [00:08<00:00, 19.22it/s]

Validation Metrics: {'accuracy': 0.8255547054322877, 'precision': 0.807817321401402, 'recall': 0.7850103890985423, 'f1_score': 0.7922461143644864, 'loss': 0.5188835238180373}
No improvement for 5 epoch(s).
Early stopping triggered. Training terminated.


In [59]:
model = KenyanFood13Model(TrainingConfiguration.num_classes)  # Replace with your actual model
model.load_state_dict(torch.load('best_model.pth'))  # Replace with your model's

# Set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Move model to the same device as inputs
model.to(device)

model.eval()  # Set to evaluation mode

# Make predictions
predictions = []
image_ids = []

with torch.no_grad():
    for images, image_names in tqdm(test_loader, desc="Making Predictions"):
        images = images.to('cuda')  # Or 'cpu' depending on your setup

        # Forward pass
        outputs = model(images)

        # Get predicted labels
        _, predicted_labels = torch.max(outputs, 1)

        # Convert numeric predictions to string labels
        predicted_labels_str = [reverse_label_map[label] for label in predicted_labels.cpu().numpy()]

        # Store predictions and image IDs
        predictions.extend(predicted_labels_str)
        image_ids.extend(image_names)

# Create a DataFrame for submission
submission_df = pd.DataFrame({
    'id': image_ids,
    'class': predictions
})

# Save to submission.csv
submission_df.to_csv('submission.csv', index=False)

print("Submission file created: submission.csv")

<ipython-input-59-b1881314b4c2>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('best_model.pth'))  # Replace with your model's
Making Pred

Submission file created: submission.csv


In [60]:
submission_df

,id,class
0,9156739011499789258,nyamachoma
1,2049465964503133373,kachumbari
2,6446998501027132988,nyamachoma
3,4194396063119815321,ugali
4,9018117998187006009,kachumbari
...,...,...
1633,18302448610371772604,githeri
1634,15920672464676076400,chapati
1635,3232020170382870007,bhaji
1636,3094804487341098468,kachumbari


In [61]:
submission_df['class'].value_counts()

,count
class,
chapati,234
nyamachoma,209
ugali,190
mandazi,156
bhaji,148
kachumbari,127
githeri,124
matoke,112
masalachips,108
